In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
from holoviews import opts
import panel as pn
import panel.widgets as pnw
from bokeh.themes.theme import Theme
pn.extension()
hv.extension('bokeh')
theme = Theme(
    json = {
    "attrs": {
        "Legend": {
            "spacing": 5,
            "glyph_width": 10,
            "label_standoff": 8,
            "label_text_color": "#FFFFFF",
            "label_text_font": "Helvetica",
            "label_text_font_size": "1.0em",
            "border_line_alpha": 0,
            "background_fill_alpha": 0.15,
            "background_fill_color": "#000000"
        },
    }
})
hv.renderer('bokeh').theme = theme
df0 = pd.read_csv('fadn_at_a_glance.csv')

In [ ]:
names = {'SE011': 'T.Hours(hrs)', 'SE025': 'T.Area(ha)', 'SE080': 'Liv.Units(LU)', 'SE131': 'T.Out.(€)',
         'SE132': 'T.Out/T.Costs (ratio)', 'SE135': 'Crop.Out (€)', 'SE136': 'Cr.Out/Ha (€/ha)', 'SE206': 'Lv.Out(€)',
         'SE207': 'Lv.Out/Lv.Un(€/LU)', 'SE256': 'Oth.Out(€)', 'SE270': 'Tot.Costs(€)', 'SE281': 'T.Sp.Cos(€)',
         'SE336': 'T.Far.Ov(€)', 'SE360': 'Depr.(€)', 'SE370': 'Wages(€)', 'SE375': 'Rent(€)', 'SE380': 'Inter.(€)',
         'SE405': 'Subs&Tax2(€)', 'SE420': 'Fam.Inc.(€)', 'SE436': 'Tot.Ass.(€)', 'SE441': 'Fixed Assets (€)', 'SE446': 'Land etc (€)',
         'SE450': 'Build.(€)', 'SE455': 'Machi(€)', 'SE460': 'Br.liv(€)', 'SE465': 'T.Cur.Ass(€)',
         'SE470': 'Non-br.liv(€)', 'SE475': 'Prod.stock', 'SE480': 'Oth.circ.(€)',
         'SE600': 'Subs&Tax1(€)', 'SE605': 'Subs.(€)'}

In [ ]:
df1 = df0.groupby(['TF14', 'COUNTRY'])['mean_perc'].mean().reset_index()
df1['mean_perc'] = df1.mean_perc.round(1)

In [ ]:
ds = hv.Dataset(df1, kdims=['TF14', 'COUNTRY'], vdims=['mean_perc']).sort()

In [ ]:
levels = [0, 1, 5, 10, 20, 30, 40, 60]
heatmap = hv.HeatMap(ds).opts(height=500, width=500, color_levels=levels, xrotation=45,tools=['hover'], 
                              title='Mean % per Country -- Click on cell', colorbar=True)

posxy = hv.streams.Tap(source=heatmap, x='(16) Specialist other fieldcrops', y='(ELL) Greece')

over = heatmap * hv.Labels(heatmap).opts(text_font_size='6pt')

country  = pnw.Select(name='Country', options=sorted(list(df0.COUNTRY.unique())))
tf  = pnw.Select(name='TF14', options=sorted(list(df0.TF14.unique())), value='(16) Specialist other fieldcrops')
def up_tf_opts(event):
    country.value, tf.value = posxy.contents['y'], posxy.contents['x']
posxy.param.watch(up_tf_opts, 'x')
posxy.param.watch(up_tf_opts, 'y')

@pn.depends(country.param.value, tf.param.value,)    
def tap_syntesis(country,tf):
    df2 = df0.query('COUNTRY == "'+ country +'" and TF14 =="' + tf +'"')
    if len(df2.index) < 2:
        return pn.Column('NO DATA AVAILABLE',pn.pane.JPG('images.jpg'))
    else:
        ds1 = hv.Dataset(df2, kdims=['TF14', 'COUNTRY'])
        cur420 =  hv.Curve(ds1,'YEAR', 'SE420', label=names['SE420'])
        cur131 =  hv.Curve(ds1,'YEAR', 'SE131', label=names['SE131'])
        cur605 =  hv.Curve(ds1,'YEAR', 'SE605', label=names['SE605'])
        cur270 =  hv.Curve(ds1,'YEAR', 'SE270', label=names['SE270'])
        curves = (cur420 * cur131 * cur605 * cur270).relabel(country + ' -- ' + tf).opts(height=250,ylabel='')
        cur425 =  hv.Curve(ds1,'YEAR', 'SE425',label='Income/AWU(€)')
        cur011 =  hv.Curve(ds1,'YEAR', 'SE011',label=names['SE011'])
        cur132 =  hv.Curve(ds1,'YEAR', 'SE132',label=names['SE132'])
        cur136 =  hv.Curve(ds1,'YEAR', 'SE136', label=names['SE136'])
        cur207 =  hv.Curve(ds1,'YEAR', 'SE207', label=names['SE207'])
        lay1 = (curves + cur425 + cur011 + cur132 + cur136 + cur207).cols(3)
        lay1.opts(opts.Curve(tools=['hover'], show_grid=True,framewise=True, width=400, height=250,
                             xticks=[year for year in range(2004,2019)], xrotation=45),
                  opts.Overlay(legend_position='top_left'))
        over1 = hv.Area(ds1, 'YEAR','plus').relabel('+POS+') * hv.Area(df2, 'YEAR','miners').opts(fill_alpha=0.5).relabel('-NEG-')
        cur1 = hv.Curve(ds1,'YEAR', 'SE420')
        over2 =  over1 * cur1.opts(line_dash='dotted', color='g')
        ar281 = hv.Area(ds1, 'YEAR','SE281',label=names['SE281'])
        ar336 = hv.Area(ds1, 'YEAR','SE336',label=names['SE336'])
        ar360 = hv.Area(ds1, 'YEAR','SE360',label=names['SE360'])
        ar370 = hv.Area(ds1, 'YEAR','SE370',label=names['SE370'])
        ar375 = hv.Area(ds1, 'YEAR','SE375',label=names['SE375'])
        ar380 = hv.Area(ds1, 'YEAR','SE380',label=names['SE380'])
        over3 = hv.Area.stack(ar281 * ar336 * ar360 * ar370 * ar375 * ar380).opts(title='-NEG-')
        ar135 = hv.Area(ds1, 'YEAR','SE135',label=names['SE135'])
        ar206 = hv.Area(ds1, 'YEAR','SE206',label=names['SE206'])
        ar256 = hv.Area(ds1, 'YEAR','SE256',label=names['SE256'])
        ar600 = hv.Area(ds1, 'YEAR','SE600',label=names['SE600'])
        ar405 = hv.Area(ds1, 'YEAR','SE405',label=names['SE405'])
        over4 =  hv.Area.stack(ar135 * ar206 * ar256 * ar600 * ar405).opts(title='+POS+')
        lay2 = over2 + over3 + over4
        lay2.opts(opts.Overlay(legend_position='top_left'),
                  opts.Area(framewise=True, ylabel='', xrotation=45, width=400))
        return pn.Column(lay1,lay2)

pn.Row(pn.Column(over,country,tf),pn.Column(tap_syntesis)).show()

